In [ ]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List

In [140]:
#create source file from .json currently in 1nf
df = pd.read_json('./data/tickets.json', lines=True)
#create a csv for d_airlines by extrating the dict values into a new table
from pandas.io.json import json_normalize

d_airline_df=pd.concat([pd.json_normalize(df['airline'][key]) for key in df.index]).reset_index(drop=True)
d_airline_df = d_airline_df.rename(columns={'name': 'airline_name', 'icao': 'airline_icao', 'iata': 'airline_iata', 'country': 'airline_country'})
d_airline_df.to_csv('./data/d_airlines.csv', mode= 'w')
#create a csv for d_origin_airport by extrating the dict values into a new table
d_origin_airport_df=pd.json_normalize(df['origin'])
d_origin_airport_df = d_origin_airport_df.rename(columns={'name': 'origin_airport_name', 'icao': 'origin_icao', 'iata': 'origin_airport_iata', 'country': 'origin_airport_country','city':'origin_airport_city'})
d_origin_airport_df.to_csv('./data/d_origin_airports.csv', mode= 'w')
#create a csv for d_passenger by extrating the dict values into a new table
d_passenger_df=pd.concat([pd.json_normalize(df['passenger'][key]) for key in df.index]).reset_index(drop=True)
#add start_date and end_date columns
d_passenger_df["start_date"] = " "
d_passenger_df["end_date"] = " "
d_origin_airport_df = d_origin_airport_df.rename(columns={'name': 'origin_airport_name'})
d_passenger_df.to_csv('./data/SCD2_d_passengers.csv', mode= 'w')
#merge datasets to make a source csv
source_df= pd.merge(d_airline_df, d_origin_airport_df, left_index=True, right_index=True)
source_df= pd.merge(source_df, d_passenger_df, left_index=True, right_index=True)
source_df.to_csv('./data/airport_source.csv')
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4096 entries, 0 to 4095
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   airline_name            4096 non-null   object 
 1   airline_iata            4096 non-null   object 
 2   airline_icao            4096 non-null   object 
 3   callsign                4096 non-null   object 
 4   airline_country         4096 non-null   object 
 5   origin_airport_name     4041 non-null   object 
 6   origin_airport_city     4041 non-null   object 
 7   origin_airport_country  4041 non-null   object 
 8   origin_airport_iata     4041 non-null   object 
 9   origin_icao             4041 non-null   object 
 10  latitude                4041 non-null   float64
 11  longitude               4041 non-null   float64
 12  altitude                4041 non-null   float64
 13  tz_timezone             3998 non-null   object 
 14  first_name              4096 non-null   

In [ ]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "../data/garys_store/files/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "receipts_1nf.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372116"
DATASET_NAME = "airline_ticket_processor"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'products': {
        'table_name': 'products',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('product_id', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('product_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('category', 'string', mode='NULLABLE'),
            bigquery.SchemaField('unit', 'string', mode='NULLABLE'),
            bigquery.SchemaField('unit_price', 'float', mode='REQUIRED'),
        ],
    },
    'customers': {
        'table_name': 'customers',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('customer_id', 'string', mode='REQUIRED'),
            bigquery.SchemaField('full_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('prefix', 'string', mode='NULLABLE'),
            bigquery.SchemaField('first_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('last_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('address', 'string', mode='REQUIRED'),
        ],
    },
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")